In [2]:
country_iso = 'BF'

In [3]:
import malariagen_data
import pandas as pd
import numpy as np
from pyprojroot import here
from myst_nb import glue
import yaml
import geopandas as gpd

In [5]:
gdf_cohorts = gpd.read_file(here() / "build" / "final_cohorts.geojson")
gdf_cohorts = gdf_cohorts.query(f"country_alpha2 == '{country_iso}'")
country_name = gdf_cohorts['country'].unique()
glue("country", country_name)
gdf_cohorts.head()

array(['Burkina Faso'], dtype=object)

,cohort_size,country,admin1_iso,admin1_name,admin2_name,taxon,year,quarter,cohort_label,sample_query,...,h12_window_size,country_alpha3,shapeName,shapeISO,shapeID,shapeGroup,shapeType,representative_lon,representative_lat,geometry
0,78,Burkina Faso,BF-09,Hauts-Bassins,Houet,coluzzii,2012,3,Burkina Faso / Houet / coluzzii / 2012 / Q3,cohort_admin2_quarter == 'BF-09_Houet_colu_201...,...,1000,BFA,Houet,,67063806B43041068804188,BFA,ADM2,-4.311852,11.389151,"POLYGON ((-3.97513 11.93914, -4.00000 11.93960..."
1,73,Burkina Faso,BF-09,Hauts-Bassins,Houet,gambiae,2012,3,Burkina Faso / Houet / gambiae / 2012 / Q3,cohort_admin2_quarter == 'BF-09_Houet_gamb_201...,...,1000,BFA,Houet,,67063806B43041068804188,BFA,ADM2,-4.311852,11.389151,"POLYGON ((-3.97513 11.93914, -4.00000 11.93960..."


# {glue:}`country`!

In [12]:
import pandas as pd
from ipyleaflet import Map, Marker, basemaps, AwesomeIcon, Popup
from ipywidgets import HTML

center = gdf_cohorts[['latitude', 'longitude']].mean().to_list()
m = Map(center=center, zoom=8, basemap=basemaps.OpenTopoMap)

for coh_id, row in gdf_cohorts.iterrows():
    lat, long = row[['latitude', 'longitude']]
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'
    
    marker = Marker(location=(lat, long), draggable=True, opacity=0.7, color=color)
    m.add_layer(marker);
    
    message2 = HTML()
    message2.value = f'<a href="https://github.com/anopheles-genomic-surveillance/selection-atlas">{coh_id}</a>'
    marker.popup = message2

display(m)

Map(center=[11.209212429169746, -4.323225424981523], controls=(ZoomControl(options=['position', 'zoom_in_text'…